In [ ]:
%pip install pandas
%pip install numpy
%pip install scikit-learn
%pip install nltk
%pip install vaderSentiment
%pip install matplotlib
%pip install seaborn   
%pip install pipeline

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Jamen Datasets(CleanerThesisPhilFakeNews (2)) VADER + LDA

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import nltk
import string
from scipy.sparse import hstack

# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('vader_lexicon')

# Load dataset
df = pd.read_csv(r"C:\Users\Dinothelo P. Quiroga\OneDrive\Desktop\fakeNews\datasets\combined\ENTERTAINMENT.csv",
                 encoding="utf-8", encoding_errors="replace", on_bad_lines="skip")

# Normalize and clean label column
if 'Label' in df.columns and df['Label'].str.lower().isin(['not credible', 'credible']).any():
    df['Label'] = df['Label'].str.title().replace({'Not Credible': 'Not Credible', 'Credible': 'Credible'})
else:
    raise ValueError("Label column missing or does not contain 'Not Credible'/'Credible' values.")

df = df.dropna(subset=['Label'])
df['combined_text'] = df['Headline'].fillna('') + " " + df['Content'].fillna('')

# Preprocessing function
def preprocess_text(text):
    if pd.isnull(text):
        return ""
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(tokens)

# Cross-validation setup
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
analyzer = SentimentIntensityAnalyzer()

accuracies = []

for fold, (train_idx, test_idx) in enumerate(skf.split(df['combined_text'], df['Label']), 1):
    print(f"\n===== Fold {fold} =====")

    # Split
    X_train_raw = df.iloc[train_idx]['combined_text']
    X_test_raw = df.iloc[test_idx]['combined_text']
    y_train = df.iloc[train_idx]['Label']
    y_test = df.iloc[test_idx]['Label']

    # Preprocess
    X_train_clean = X_train_raw.apply(preprocess_text)
    X_test_clean = X_test_raw.apply(preprocess_text)

    # TF-IDF
    tfidf_vectorizer = TfidfVectorizer(max_features=5000)
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_clean)
    X_test_tfidf = tfidf_vectorizer.transform(X_test_clean)

    # VADER Sentiment
    X_train_sentiment = np.array(X_train_clean.apply(lambda x: analyzer.polarity_scores(x)['compound'])).reshape(-1, 1)
    X_test_sentiment = np.array(X_test_clean.apply(lambda x: analyzer.polarity_scores(x)['compound'])).reshape(-1, 1)

    # Combine TF-IDF + VADER
    X_train_features = hstack([X_train_tfidf, X_train_sentiment])
    X_test_features = hstack([X_test_tfidf, X_test_sentiment])

    # LDA (convert to dense)
    lda = LinearDiscriminantAnalysis(n_components=1)
    X_train_lda = lda.fit_transform(X_train_features.toarray(), y_train)
    X_test_lda = lda.transform(X_test_features.toarray())

    # Train
    svm = SVC(kernel='linear', C=1.0, random_state=42)
    svm.fit(X_train_lda, y_train)

    # Evaluate
    y_pred = svm.predict(X_test_lda)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=['Not Credible', 'Credible']))

# Final average accuracy
print(f"\n==== Cross-Validation Summary ====")
print(f"Mean Accuracy: {np.mean(accuracies):.4f}")
print(f"All Fold Accuracies: {['{:.4f}'.format(a) for a in accuracies]}")


[nltk_data] Downloading package stopwords to C:\Users\Dinothelo P.
[nltk_data]     Quiroga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Dinothelo P.
[nltk_data]     Quiroga/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Dinothelo P.
[nltk_data]     Quiroga/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Dinothelo P.
[nltk_data]     Quiroga/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!



Accuracy: 0.9528
Classification Report:
               precision    recall  f1-score   support

Not Credible       0.95      0.95      0.95      1022
    Credible       0.95      0.95      0.95      1010

    accuracy                           0.95      2032
   macro avg       0.95      0.95      0.95      2032
weighted avg       0.95      0.95      0.95      2032



with INPUT and OUTPUT of Credible or Not and Confidence Score

In [28]:
from sklearn.calibration import CalibratedClassifierCV

# Calibrate the SVM model for proper probability outputs
svm_model = SVC(kernel='linear', C=1.0, class_weight='balanced', probability=True, random_state=42)
svm_model.fit(X_train, y_train)  # Retrain with probability=True

# Input and Prediction Function
def predict_news(text):
    if not text.strip():
        print("Input is empty. Please enter valid news text.")
        return
    
    # Preprocess input
    text_clean = preprocess_text(text)
    token_count = len(text_clean.split())
    
    # Check for gibberish or too short input
    if token_count < 5:
        print("Input too short or unclear. Please provide a longer news article.")
        return
    
    # Transform with the same TF-IDF vectorizer
    text_tfidf = tfidf_vectorizer.transform([text_clean])
    
    # Sentiment analysis
    sentiment_score = get_sentiment_score(text_clean)
    sentiment_score_array = np.array([[sentiment_score]])
    
    # Combine features
    text_features = hstack([text_tfidf, sentiment_score_array])
    
    # Convert to dense
    text_features_dense = text_features.toarray()
    
    # Apply LDA transformation
    text_reduced = lda.transform(text_features_dense)
    
    # Prediction and Confidence
    pred_prob = svm_model.predict_proba(text_reduced)[0]
    pred = np.argmax(pred_prob)
    confidence = pred_prob[pred] * 100
    
    label = 'Real News' if pred == 1 else 'Fake News'
    
    print(f"\nPrediction: {label}")
    print(f"Confidence Score: {confidence:.2f}%")

# Example usage
sample_text = input("Enter the news article text: ")
predict_news(sample_text)



Prediction: Real News
Confidence Score: 96.02%
